In [13]:
import numpy as np
import time
import keras
import pandas as pd
from sklearn.model_selection import train_test_split

In [14]:
X = np.squeeze(np.load('src/smiley_X.npy'))
y = np.load('src/smiley_y.npy')
y = keras.utils.to_categorical(y-1, num_classes = 2)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)


In [16]:
x_train = x_train.reshape(115, 9, 9, 1)
x_test = x_test.reshape(29, 9, 9, 1)
x_train.shape #28*28=784

(115, 9, 9, 1)

In [17]:
model = keras.models.Sequential([
  keras.layers.Conv2D(32, (2,2), activation='relu', input_shape=(9, 9, 1)),
  keras.layers.MaxPooling2D(2, 2),
  keras.layers.Conv2D(32, (2,2), activation='relu'),
  keras.layers.MaxPooling2D(2,2),
  keras.layers.Flatten(),
  keras.layers.Dense(8, activation='relu'),
  keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=[keras.metrics.categorical_accuracy, keras.metrics.TruePositives(), keras.metrics.FalsePositives(), keras.metrics.TrueNegatives(), keras.metrics.FalseNegatives(), keras.metrics.Precision(), keras.metrics.Recall()])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 8, 8, 32)          160       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 32)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 3, 32)          4128      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 1, 1, 32)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                

In [18]:
model.fit(x_train,y_train, epochs=5)
model.evaluate(x_test,y_test)

Epoch 1/5
4/4 [==============================] - 2s 4ms/step - loss: 0.6909 - categorical_accuracy: 0.4696 - true_positives_1: 54.0000 - false_positives_1: 61.0000 - true_negatives_1: 54.0000 - false_negatives_1: 61.0000 - precision_1: 0.4696 - recall_1: 0.4696
Epoch 2/5
4/4 [==============================] - 0s 3ms/step - loss: 0.6798 - categorical_accuracy: 0.6087 - true_positives_1: 70.0000 - false_positives_1: 45.0000 - true_negatives_1: 70.0000 - false_negatives_1: 45.0000 - precision_1: 0.6087 - recall_1: 0.6087
Epoch 3/5
4/4 [==============================] - 0s 3ms/step - loss: 0.6711 - categorical_accuracy: 0.6174 - true_positives_1: 71.0000 - false_positives_1: 44.0000 - true_negatives_1: 71.0000 - false_negatives_1: 44.0000 - precision_1: 0.6174 - recall_1: 0.6174
Epoch 4/5
4/4 [==============================] - 0s 4ms/step - loss: 0.6605 - categorical_accuracy: 0.5826 - true_positives_1: 67.0000 - false_positives_1: 48.0000 - true_negatives_1: 67.0000 - false_negatives_1: 4

[0.6175036430358887,
 0.6206896305084229,
 18.0,
 11.0,
 18.0,
 11.0,
 0.6206896305084229,
 0.6206896305084229]

# Experiment With Various Parameters That Control The Learning
Configuration Suite

In [19]:
X = X.reshape(144,9,9,1)

optimisations = [keras.optimizers.SGD, keras.optimizers.RMSprop, keras.optimizers.Adam, keras.optimizers.Nadam]
epochs = [3, 5, 10]
η_list = [0.1, 0.01, 0.001]
layer_list = [[64,16,2],[32,8,2],[16,4,2]]
pooling_list = [(1,1),(2,2)]
activations = ["sigmoid", "tanh", "relu", "LeakyReLU"]

In [20]:
from sklearn.model_selection import KFold

run_config_suit = False
file_name = "df_CNN.pkl"
kf = KFold(n_splits = 10)

if run_config_suit:
    start = time.time()
    list_of_results = []
    list_of_crossedevaluated = []
    for act in activations:
        for η in η_list:
            for layer in layer_list:
                for opt in optimisations:
                    for pooling in pooling_list:
                        print("\n####################################################################################")
                        print("Layers    Pool  Eta  Act   <Optimisation>")
                        print(f"{layer} {pooling} {η} {act} {opt}")
                        print("Epoch: [Loss, Accuracy, TP, FP, TN, FN, Precision, Recall]")
                        for epoch in epochs:
                            model = keras.models.Sequential([
                                keras.layers.Conv2D(layer[0], (2,2), activation=act, input_shape=(9, 9, 1)),
                                keras.layers.MaxPooling2D(pooling[0],pooling[1]),
                                keras.layers.Conv2D(layer[0], (2,2), activation=act),
                                keras.layers.MaxPooling2D(pooling[0],pooling[1]),
                                keras.layers.Flatten(),
                                keras.layers.Dense(layer[1], activation=act),
                                keras.layers.Dense(2, activation='softmax')
                            ])
                            model.compile(loss=keras.losses.categorical_crossentropy,
                                                  optimizer=opt(learning_rate=η),
                                                  metrics=[keras.metrics.categorical_accuracy, keras.metrics.TruePositives(), keras.metrics.FalsePositives(), keras.metrics.TrueNegatives(), keras.metrics.FalseNegatives(), keras.metrics.Precision(), keras.metrics.Recall()])

                            for train, test in kf.split(X):
                                model.fit(X[train],y[train], epochs=epoch, verbose= 0)
                                result = model.evaluate(X[test],y[test], verbose= 0)
                                list_of_crossedevaluated.append(result)

                            config = [opt, η, epoch, act, pooling, layer]

                            loss = 0
                            acc = 0
                            TP = 0
                            FP = 0
                            TN = 0
                            FN = 0
                            Precision = 0
                            Recall = 0
                            for i in list_of_crossedevaluated:
                                loss += i[0]
                                acc += i[1]
                                TP += i[2]
                                FP += i[3]
                                TN += i[4]
                                FN += i[5]
                                Precision += i[6]
                                Recall += i[7]
                            result10 = [loss,acc,TP,FP,TN,FN,Precision,Recall]
                            result = []
                            for i in range(2):
                                result.append(result10[i]/10)
                            result.append(TP)
                            result.append(FP)
                            result.append(TN)
                            result.append(FN)
                            for i in range(6,8):
                                result.append(result10[i]/10)
                            list_of_results.append(result + config)
                            list_of_crossedevaluated.clear()
                            print(f"\t{epoch}: {result}")

    print("\n\n############## DONE")
    print(time.time() - start)

    labels = ["Loss", "Accuracy", "TP", "FP", "TN", "FN", "Precision", "Recall", "optimiser", "η", "epoch", "activation function", "pooling", "layers"]
    dfcv = pd.DataFrame(data = list_of_results, columns=labels)
    dfcv.to_pickle(file_name)

else:
    dfcv = pd.read_pickle(file_name)

It took 3518s process (58.5 mins) to run this

Overall 864 models were generated with different hyperparameters.
layers_list = 3 variations
η_list = 3 variations
epochs = 3 variations
optimisations = 4 variations
activations = 4 variations
pooling = 2 variations
3 x 3 × 3 × 4 × 4 × 2 = 864.
The dataframe holds the metrics for each configuration and the configuration details.

In [21]:
dfcv

,Loss,Accuracy,TP,FP,TN,FN,Precision,Recall,optimiser,η,epoch,activation function,pooling,layers
0,0.745721,0.460000,66.0,78.0,66.0,78.0,0.460000,0.460000,<class 'keras.optimizers.optimizer_v2.gradient...,0.100,3,sigmoid,"(1, 1)","[64, 16, 2]"
1,0.706503,0.514286,74.0,70.0,74.0,70.0,0.514286,0.514286,<class 'keras.optimizers.optimizer_v2.gradient...,0.100,5,sigmoid,"(1, 1)","[64, 16, 2]"
2,0.730689,0.471429,68.0,76.0,68.0,76.0,0.471429,0.471429,<class 'keras.optimizers.optimizer_v2.gradient...,0.100,10,sigmoid,"(1, 1)","[64, 16, 2]"
3,0.745533,0.465714,67.0,77.0,67.0,77.0,0.465714,0.465714,<class 'keras.optimizers.optimizer_v2.gradient...,0.100,3,sigmoid,"(2, 2)","[64, 16, 2]"
4,0.750699,0.500000,72.0,72.0,72.0,72.0,0.500000,0.500000,<class 'keras.optimizers.optimizer_v2.gradient...,0.100,5,sigmoid,"(2, 2)","[64, 16, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0.048675,0.986667,142.0,2.0,142.0,2.0,0.986667,0.986667,<class 'keras.optimizers.optimizer_v2.nadam.Na...,0.001,5,LeakyReLU,"(1, 1)","[16, 4, 2]"
860,0.038046,1.000000,144.0,0.0,144.0,0.0,1.000000,1.000000,<class 'keras.optimizers.optimizer_v2.nadam.Na...,0.001,10,LeakyReLU,"(1, 1)","[16, 4, 2]"
861,0.576494,0.879048,126.0,18.0,126.0,18.0,0.879048,0.879048,<class 'keras.optimizers.optimizer_v2.nadam.Na...,0.001,3,LeakyReLU,"(2, 2)","[16, 4, 2]"
862,0.338526,0.899524,129.0,15.0,129.0,15.0,0.899524,0.899524,<class 'keras.optimizers.optimizer_v2.nadam.Na...,0.001,5,LeakyReLU,"(2, 2)","[16, 4, 2]"


Filter for 0 loss
- 5  epochs
- 10 epochs

The lower the loss the better so 0 is the most desirable. Same is true for epochs. The fewer epochs it has to run the better it is.

In [22]:
dfcv[dfcv.Loss == 0 ]

,Loss,Accuracy,TP,FP,TN,FN,Precision,Recall,optimiser,η,epoch,activation function,pooling,layers
446,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,10,relu,"(1, 1)","[64, 16, 2]"
494,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,10,relu,"(1, 1)","[16, 4, 2]"
542,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.01,10,relu,"(1, 1)","[32, 8, 2]"
655,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.10,5,LeakyReLU,"(1, 1)","[64, 16, 2]"
656,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.10,10,LeakyReLU,"(1, 1)","[64, 16, 2]"
662,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,10,LeakyReLU,"(1, 1)","[64, 16, 2]"
666,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.nadam.Na...,0.10,3,LeakyReLU,"(1, 1)","[64, 16, 2]"
667,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.nadam.Na...,0.10,5,LeakyReLU,"(1, 1)","[64, 16, 2]"
685,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,5,LeakyReLU,"(1, 1)","[32, 8, 2]"
710,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,10,LeakyReLU,"(1, 1)","[16, 4, 2]"


# Overall Best:

In [23]:
dfcv[dfcv.Loss == 0 ][dfcv.epoch == 5]

C:\Users\barth\AppData\Local\Temp\ipykernel_9404\2342856153.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfcv[dfcv.Loss == 0 ][dfcv.epoch == 5]


,Loss,Accuracy,TP,FP,TN,FN,Precision,Recall,optimiser,η,epoch,activation function,pooling,layers
655,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.10,5,LeakyReLU,"(1, 1)","[64, 16, 2]"
667,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.nadam.Na...,0.10,5,LeakyReLU,"(1, 1)","[64, 16, 2]"
685,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,5,LeakyReLU,"(1, 1)","[32, 8, 2]"
733,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.01,5,LeakyReLU,"(1, 1)","[64, 16, 2]"


# Second Best:

In [24]:
dfcv[dfcv.Loss == 0 ][dfcv.epoch == 10]

C:\Users\barth\AppData\Local\Temp\ipykernel_9404\2436025107.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfcv[dfcv.Loss == 0 ][dfcv.epoch == 10]


,Loss,Accuracy,TP,FP,TN,FN,Precision,Recall,optimiser,η,epoch,activation function,pooling,layers
446,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,10,relu,"(1, 1)","[64, 16, 2]"
494,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,10,relu,"(1, 1)","[16, 4, 2]"
542,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.01,10,relu,"(1, 1)","[32, 8, 2]"
656,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.rmsprop....,0.10,10,LeakyReLU,"(1, 1)","[64, 16, 2]"
662,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,10,LeakyReLU,"(1, 1)","[64, 16, 2]"
710,0.0,1.0,144.0,0.0,144.0,0.0,1.0,1.0,<class 'keras.optimizers.optimizer_v2.adam.Adam'>,0.10,10,LeakyReLU,"(1, 1)","[16, 4, 2]"


# Conclusion:

We have a higher number of different configuration reaching 0 loss in 5 epoch using convolutional neural network than classic ANN.
Among the best results we have:
- __Nodes Per Layers__: `[64, 16, 2]`. This was the most frequent out of the best configurations. With also one appearance of `[32, 8, 2]`.
- __Optimiser__: `Adam`, `RMSprop` and  `Nadam`.
- __η__: `0.1` and `0.01`.
- __Activation Function__ is in most case `LeakyReLU`.
- __Max_pooling__ is `(1,1)`.
